In [1]:
# DATA2001 Week 5 Tutorial
# Material last updated: 24 Mar 2025
# Note: this notebook was designed with the Roboto Condensed font, which can be installed here: https://www.1001fonts.com/roboto-condensed-font.html

from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Week 5 - Indexes and Data Partitioning

Our final week featuring the NSW FuelCheck dataset - this time to get a sense of the impacts database indexes can have. We'll also use it as an opportunity to demonstrate the generation of synthetic data through some more advanced [DDL](https://en.wikipedia.org/wiki/Data_definition_language).

## 1. Introduction

Largely a recap, plus one additional dataset provided on Canvas.

### 1.1. Configuration

Our helper functions from last week's tutorial - one to connect to our localhost database, the other to query it. Remember this requires an adjusted `Credentials.json` file in the same location as this notebook!

In [ ]:
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
db, conn = pgconnect(credentials)
conn.execute(text("set search_path to NSWFuel"))

### 1.2. Ingestion

Our new `Trips.csv` dataset contains the *essential* information for three potential road-trips - one north, one south, and one west...all revolving around 7-Eleven stores. It details, on each highway, in what order what stations would be reached, and how far they are apart. It's a small, simple CSV provided on Canvas, which, when downloaded, can be ingested to our database:

In [ ]:
conn.execute(text("""
DROP TABLE IF EXISTS Trips CASCADE;
CREATE TABLE Trips(
   Station VARCHAR(50) PRIMARY KEY,
   Highway VARCHAR(50),
   Stop INTEGER,
   Minutes FLOAT,
   Kilometres FLOAT,
   TotalKM FLOAT
);"""))
tripdata = pd.read_csv('Trips.csv')
tripdata.columns = map(str.lower, tripdata.columns)
tripdata.to_sql("trips", con=conn, if_exists='append', index=False)
query(conn, "select * from Trips limit 10")

### 1.3. Exploration

Let's again kick off by exploring the new dataset, via a few introductory tasks.

**Task 1a: Calculate the total time, distance, and average speed, of each highway journey.**

Recall that speed is calculated as distance/time.

In [ ]:
### TO DO
sql = """

"""
query(conn, sql)

**Task 1b: Select one highway journey of your choice. Which station has the most dates for which observations were made?**

List all stations in descending order of number of dates recorded. Include each station's name and address in your output.

In [ ]:
### TO DO
sql = """

"""
query(conn, sql)

**Task 1c: For your selected highway journey, AND a selected fuel type, return some summary statistics about prices at each of the stations on the trip.**

e.g. use FuelCode = 'E10' on the 'Great Western', and return how many price observations there are at each station, alongside the lowest, average and highest prices. Order your output by the "Stop" field so they are in order.

Do prices differ significantly? Is there a particular station that would be most worthwhile fuelling up at?

In [ ]:
### TO DO
sql = """

"""
query(conn, sql)

## 2. Generated Data

As a final add-in for our dataset, we'll introduce some **randomly generated data**, produced programmatically, and meaning that every person's version will differ from those around them.

### 2.1. Generation

As described in the slide deck for this week's tutorial, the code below will create two tables:

- **SimulatedDrivers:** A hypothetical list of 1000 people (_insert [fictitious persons disclaimer](https://en.wikipedia.org/wiki/Fictitious_persons_disclaimer)_)
- **SimulatedTrips:** A supposed log of these people undertaking the 7-Eleven Pilgrimage, to various extents, at various speeds

This is achieved with a single file, `TripSimulations.sql`, which you'll find on Canvas, and can be read in as below. Note that it involves a complicated query or two that do not well reflect the difficulty expected of SQL code in this course - so don't stress if a lot of it seems overly complex!

In [ ]:
with open('TripSimulations.sql', 'r') as f:
    simulationsquery = f.read()
print(simulationsquery)

#### Which aspects of this SQL query can you successfully interpret?

Let's try executing the query. Try running the cell below two or three times, and you'll notice the output changes each time.

In [ ]:
conn.execute(text(simulationsquery))
query(conn, "select * from SimulatedDrivers limit 10")

Exploring the other table of the two, you'll begin to get a sense of the simulated trip log, which records for each trip, who the driver was, what stations they stopped at, and at what time:

In [ ]:
query(conn, "select * from SimulatedTrips limit 10")

### 2.2. Exploration

**Task 2a: For each trip, calculate how many stops were made, and the earliest and latest times recorded.**

In [ ]:
### TO DO
sql = """

"""
query(conn, sql)

**Task 2b: For each trip with at least two stops, calculate the total distance travelled and the total minutes of the journey.**

Extracting an "[`epoch`](https://www.postgresql.org/docs/8.1/functions-datetime.html)" may be one of the most useful approaches to calculating the total time, and the `Trips` table will be necessary to include the distances.

In [ ]:
### TO DO
sql = """

"""
query(conn, sql)

**Task 2c: Fines are to be issued to all drivers who exceeded an average speed of 110km/h. Calculate the speeds of all trips, and return the driver name, car model, start time and speed of all drivers who exceeded this limit in the overall trip.**

Make sure the calculated speed exists in a column named "speed", for the cells that follow afterwards.

In [ ]:
### TO DO
sql = """

"""
query(conn, sql)

### 2.3. Database Views

It's worth quickly mentioning **database views**, which can prove a helpful means of storing the calculation logic for a query, without needing to store the materialised results at a point in time.

Take the query we just produced above for the speed offenders, for example. Paste your query into the cell below (removing the speed limit of 110, and any `order by` clause, if you have one), and you should be able to save your query in your localhost database for future reference as a _view_, not a table, titled "SimulatedSummary":

In [ ]:
### TO DO
sql = """
drop view if exists SimulatedSummary;
create view SimulatedSummary as

/* INSERT QUERY HERE */

;
select * from SimulatedSummary limit 5
"""
query(conn, sql)

We could then call that view whenever needed, and apply any filtering as needed, for example recreating the speed offenders:

In [ ]:
sql = """
select * from SimulatedSummary where speed > 110 order by speed desc
"""
query(conn, sql)

## 3. Indexes

As covered in the lecture and the tutorial slide deck, indexes can prove a helpful means of optimising the process of querying a database. Let's explore with some examples based on our work above, and see if we can assess the impact. Note that some indexes will already exist - try running the query below and consider its implications:

In [ ]:
sql = """
select * from pg_indexes
"""
query(conn, sql)

### 3.1. Exploration

Complete the following tests in your pgAdmin window (making sure to begin with `set search_path to NSWFuel;`).

**Task 3a: Run each query below, one at a time, and observe the planning and execution times (write them down). Do they vary?**

In [ ]:
statements = """
explain analyse select * from SimulatedDrivers where family_name = 'Jones';
explain analyse select count(*) from SimulatedDrivers where vehicle_year < 2005;
explain analyse select * from SimulatedTrips where extract(hour from stop_time) < 6;
explain analyse select * from SimulatedSummary where speed > 110;
"""

**Task 3b: Run all commands below, then return to each of the queries from above, again one at a time. Comparing the planning and execution times to what you previously recorded, have the indexes had much of an impact? Why might this be the case? Are any indexes redundant?**

If you'd like to drop the indexes to re-test the "before" state, use the second cell included below, which simply removes them.

You can also run the `VACUUM ANALYSE` command to clear cache, as needed.

In [ ]:
indexes = """
create index simulated_surname_ind on SimulatedDrivers(family_name);
create index simulated_year_ind on SimulatedDrivers(vehicle_year);
create index simulated_trip_ind on SimulatedTrips(trip_id);
create index simulated_hour_ind on SimulatedTrips(extract(hour from stop_time));
"""

In [ ]:
dropcommands = """
drop index if exists simulated_surname_ind;
drop index if exists simulated_year_ind;
drop index if exists simulated_trip_ind;
drop index if exists simulated_hour_ind;
"""

**Task 3c: Consider the impact data partitioning could have on a dataset of this nature. Give an example of a way in which partitioning could be applied, that would be of benefit to a query you can imagine of interest.**

In [ ]:
# no code required

#### Complete!

That concludes this week's tutorial. We'll continue next week with an introduction to web scraping, as a segue into the world of semi-structured data. Enjoy your week!